In [57]:
#Natural Language Tool Kit
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [58]:
with open('artigos.txt', "r") as f:
  artigos = f.read()

In [59]:
tokens = nltk.tokenize.word_tokenize(artigos) #Tokenização

In [60]:
#removendo pontuações
def remove_pontuação(tokens):
  tokens_so_palavras = []
  for token in tokens:
    if token.isalpha():
      tokens_so_palavras.append(token)
  return tokens_so_palavras
tokens = remove_pontuação(tokens)




In [61]:
#Normalização
def normalização(tokens):
  tokens_normalizados =[]
  for token in tokens:
    tokens_normalizados.append(token.lower())
  return  tokens_normalizados
tokens_normalizados = normalização(tokens)

In [62]:
#Remover palavras repetidas
tokens_normalizados_unicos = set(tokens_normalizados)

In [63]:
#Fatiamento de palavras e manipulação das palavras
def insere_letras(fatias):
  novas_palavras = []
  letras = "abcdefghijklmnopqrstuvwxyzáàéíóúâêîôûãõç"
  for E,D in fatias:
    for letra in letras:
      novas_palavras.append(E + letra + D)
  return novas_palavras

def deleta_caracter(fatias):
  novas_palavras = []
  for E,D in fatias:
    novas_palavras.append(E + D[1:])
  return novas_palavras


def troca_letra(fatias):
  novas_palavras = []
  letras = "abcdefghijklmnopqrstuvwxyzáàéíóúâêîôûãõç"
  for E,D in fatias:
    for letra in letras:
      novas_palavras.append(E + letra + D[1:])
  return novas_palavras

def inverte_letra(fatias):
  novas_palavras = []
  for E,D in fatias:
    if len(D) > 1:
      novas_palavras.append(E +D[1] + D[0]+ D[2:])
  return novas_palavras


In [91]:
#gerador de palavras posiveis
def gerador_de_palavras(palavra):
  fatias =[]
  for i in range(1 + len(palavra)):
    fatias.append((palavra[:i],palavra[i:]))
  palavras_geradas = insere_letras(fatias)
  palavras_geradas += deleta_caracter(fatias)
  palavras_geradas += troca_letra(fatias)
  palavras_geradas += inverte_letra(fatias)
  return palavras_geradas

def gerador_aprimorado(palavras_geradas):
  novas_palavras=[]
  for palavra in palavras_geradas:
    novas_palavras += gerador_de_palavras(palavra)
  return novas_palavras

In [122]:
total_de_palavras = len(tokens_normalizados)
frequencia = nltk.FreqDist(tokens_normalizados)
def corretor(palavra):
  palavras_geradas = gerador_de_palavras(palavra)
  palavra_correta = max(palavras_geradas,key=propabilidade)
  return palavra_correta

def propabilidade(palavra_gerada):
  return frequencia[palavra_gerada]/total_de_palavras

def corretor_aprimorado(palavra):
  palavras_geradas = gerador_de_palavras(palavra)
  palavras_geradas_aprimoradas = gerador_aprimorado(palavras_geradas)
  soma_palavras = set(palavras_geradas + palavras_geradas_aprimoradas)
  palavras_conhecidas=[]
  for palavra in soma_palavras:
    if palavra in vocabulario:
      palavras_conhecidas.append(palavra)
  try:
    palavra_correta = max(palavras_conhecidas,key=propabilidade)
  except:
    palavra_correta = palavra
  return palavra_correta

def corretor_condicional(palavra):
  palavras_geradas = gerador_de_palavras(palavra)
  suposta_palavra_correta = max(palavras_geradas,key=propabilidade)
  palavras_conhecidas=[]
  if suposta_palavra_correta in vocabulario:
    palavra_correta = suposta_palavra_correta
  else:
    palavras_geradas_aprimoradas = gerador_aprimorado(palavras_geradas)
    soma_palavras = set(palavras_geradas + palavras_geradas_aprimoradas)
    for palavra in soma_palavras:
      if palavra in vocabulario:
         palavras_conhecidas.append(palavra)
    try:
      palavra_correta = max(palavras_conhecidas,key=propabilidade)
    except:
      palavra_correta = palavra
  return palavra_correta
  


In [118]:
def avaliação(testes, vocabulario):
  numero_palavras = len(testes)
  acertou = 0
  desconhecida = 0
  for correta, errada in testes:
    palavra_corrigida = corretor_condicional(errada)
    desconhecida +=(correta not in vocabulario)
    if palavra_corrigida == correta:
      acertou += 1    
  taxa_acerto = round(100*acertou/numero_palavras,2)
  taxa_desconhecida = round(100*desconhecida/numero_palavras,2)
  return taxa_acerto, taxa_desconhecida

In [67]:
def dados_teste(arquivo):
  lista_de_palavras_teste =[]
  f = open(arquivo,"r")
  for linha in f:    
    correta, errada =linha.split()
    lista_de_palavras_teste.append((correta, errada))
  f.close
  return lista_de_palavras_teste


In [104]:
lista_teste = dados_teste("palavras.txt")
vocabulario = tokens_normalizados_unicos

In [123]:
avaliação(lista_teste, vocabulario)

(83.87, 6.99)